In [1]:
import sys
import os
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re

import src.io as sio
import src.qudi_fit_wrapper as qft
import src.helpers as shp
import src.fitting as sft

from scipy.optimize import curve_fit

plt.style.use('seaborn')

In [2]:
DATA_FOLDER = "20220405_FR0612-F2-3G6"
DATA_FOLDERPATH, FIGURE_FOLDERPATH = sio.get_data_and_figure_paths(DATA_FOLDER)

src.io :: INFO :: qudiamond folderpath is \\kernix\qudiamond\Data\20220405_FR0612-F2-3G6\


In [3]:
os.listdir(DATA_FOLDERPATH)

['2022',
 'frq-sweep-qnami-after-bonding001.dat',
 'frq-sweep-qnami-before-bonding001.dat',
 'LaserNormalizing.csv',
 'LaserNormalizing.xlsx',
 'Nanonis-Session.ini']

In [ ]:
df = pd.read_csv(os.path.join(DATA_FOLDERPATH, "LaserNormalizing.csv"))
df['Obj Power (uW)'] = df['Obj Power (mW)'] * 1e3
df = df.sort_values("")

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))

ax.plot(df['Obj Power (mW)'] * 1e3, df['Counts'], "o")
ax.set_ylabel("Fluorescence (kcps)")
ax.set_xlabel("Power at objective (μW)")
ax.set_title("FR0612-F2-3G6")
fig.tight_layout()

# sio.save_figures("laser_normalization", folder=FIGURE_FOLDERPATH)

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['GUI power (mW)'], df['Obj Power (uW)'], "o")
popt, pcov = curve_fit(linear, df['GUI power (mW)'], df['Obj Power (uW)'])
ax.plot(df['GUI power (mW)'], linear(df['GUI power (mW)'], *popt))
ax.set_ylabel("Power at objective (uW)")
ax.set_xlabel("Power in GUI (mW)")
print()

In [ ]:
def linear(x, a, b):
    return a + b * x

def logarithm(x, a, b):
    return a + b * np.log(x)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

ax.plot(df['Obj Power (uW)'], df['Counts'], "o")

# Log fit
popt, pcov = curve_fit(logarithm, df['Obj Power (uW)'], df['Counts'])
x = np.linspace(df['Obj Power (uW)'].min(), df['Obj Power (uW)'].max(), 100)
ax.plot(x, logarithm(x, *popt), label="Log fit to all data")

# Linear fit optimizer
# for b in range(4, 9, 1):
#     popt, pcov = curve_fit(linear, df['Obj Power (uW)'][0:b], df['Counts'][0:b])
#     ax.plot(df['Obj Power (uW)'][0:9], linear(df['Obj Power (uW)'][0:9], *popt), label=f"b={b}")

# Linear fit result
optimal_point = 6
popt, pcov = curve_fit(linear, df['Obj Power (uW)'][0:optimal_point], df['Counts'][0:optimal_point])
ax.plot(df['Obj Power (uW)'], linear(df['Obj Power (uW)'], *popt), label="Linear fit after saturation")
ax.axvline(df['Obj Power (uW)'][optimal_point - 1], linestyle="--", label="Estimated saturation point")

optimal_point = 3
popt, pcov = curve_fit(linear, df['Obj Power (uW)'][-optimal_point:], df['Counts'][-optimal_point:])
ax.plot(df['Obj Power (uW)'][-optimal_point - 1:], linear(df['Obj Power (uW)'][-optimal_point - 1:], *popt), label="Linear fit after saturation")

ax.axvline(df['Obj Power (uW)'][len(df['Obj Power (uW)']) - optimal_point - 1], linestyle="-.", label="Max curvature point")

ax.set_ylabel("Fluorescence (kcps)")
ax.set_xlabel("Power at objective (μW)")
# ax.set_title("FR0612-F2-3G6")
ax.legend()

ax2 = ax.twiny()
ax2.plot(df['GUI power (mW)'], df['Counts'], " ")
ax2.set_xlabel("Power in GUI (mW)")

fig.tight_layout()

sio.save_figures("saturation_point", folder=FIGURE_FOLDERPATH)

In [ ]:
df['GUI power (mW)'].sort_values()

# T1

In [4]:
t1 = sio.read_pulsed_measurement_dataclass(DATA_FOLDERPATH, measurement_str="t1")

\\kernix\qudiamond\Data\20220405_FR0612-F2-3G6\2022\04\20220414: : 23it [00:01, 16.62it/s]                


In [5]:
t1

OrderedDict()